In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [39]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator


In [40]:
tf.__version__

# Part 1 - Data Preprocessing

### Preprocessing the Training Set


Transformation on all images - to avoid overfitting 
 If we don't apply those transformations, when training the CNN on the training set  we will get a huge difference between the accuracy on the training set
and the one on the test set

On the evaluation set, we will get very high accuracy on the training set, close to 98 % a
 and much lower accuracy on the test - called "overfitting"

 - What are those transformations: it is some simple geometrical transformations
 or some zooms or some rotation on the images, horizontal flips

 We are going to do a series of transformation to modify the images and get them augmented

- Image Augmentation - which consists of transforming our images of the training set so that   your CNN model does not overlearned , is not overtrained on the existing images. because by applying those transformations we will get new images  we augment the diversity of the training set images

- We are going to use from ImageDataGenerator: sheer_range, zoom_range and horizontal_flip

In [41]:
train_datagen = ImageDataGenerator(
        rescale=1./255, #we get all the pixel values between 0 and 1
        shear_range=0.2, 
        zoom_range=0.2,
        horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        '../input/dogs-cats-images/dataset/training_set',
        target_size=(64, 64),# final size of our images when they will be fed into CNN
        batch_size=32,# how many images we want to have in each batch, 32 is default value
        class_mode='binary') # binary outcome, cat or dog

# connect the dataset

#import the training set, and creating batches and resizing the images

train_datagen is an instance of that image data generator clas in which represents
the tool that will apply all the transformations on the images of the training set

- sheer_range: Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
- zoom_range: Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range].
-  horizontal_flip:Boolean. Randomly flip inputs horizontally.

 rescale: rescaling factor. Defaults to None. 
If None or 0, no rescaling is applied,
#otherwise we multiply the data by the value provided (after applying all other transformations). it will apply feature scaling to each and every single one of your pixels by dividing their value by 255. just like normalization

### Preprocessing the Test Set


In [42]:
test_datagen = ImageDataGenerator(rescale=1./255)
#we don't want touch test images but we need to rescale
#no transformation for the test

test_set = test_datagen.flow_from_directory(
        '../input/dogs-cats-images/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

# Part 2 - Building the CNN

### Initialising the CNN

In [43]:
cnn=tf.keras.models.Sequential()

## Step 1 - Convolution

In [44]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation = "relu",
                               input_shape=[64,64,3]))



add the first convolutional level. 

Arguments:
- filters - Integer, the dimensionality of the output space (the number of output filter in the convolution) feature detectors. Classic Architecture: 
    First Convolutional layer - 32, Second convolutional layer : 32
- kernel_size - An inteeger or tuple of 2 integers, specifying the height and width of the 2D Convolution window, the number of rows and columns of Feature Detector (3x3) 
-no default value of the activation, because on the general rule, as long as we have not reached the output layer, we rather want to get a rectified activation function.We will use the "RELU"(rectifier activation function) parameters
- input_shape - when you add your very first layer, whether it is convolutional or a dense layer, you have to specify the input shape of your inputs. (64,64,3) for colorful images, (64,64,1) - if we worked with black-and-white images

 ## Step 2 - Pooling

In [45]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

We will apply a max pooling to obtain the pooled feature map.

Arguments:
- pool_size (2x2 square)
- strides (recommended: 2) sliding two by two pixels
- padding - when we reach the border with extra two empty cells, with a valid padding, we will just ignore empty cells, and with the same padding, we will add an extra column with fake pixels that are equal to 0

## Add a second convolutional layer

In [46]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation = "relu"))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
we do flatten results (of pulling and convolutions) into one dimensional vector

In [47]:
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection

In [48]:
cnn.add(tf.keras.layers.Dense(units=128, activation = "relu"))

Arguments:
- units - number of hidden neurons (128) have a larger number of neurons
- activation - rectifier activation function

# Step 5 - Output Layer

In [49]:
cnn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))
#binary classification ( 0 or 1 )

# Part 3 - Training the CNN

## Compiling the CNN

In [50]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the CNN on the Training Set and evaluating it on the Test Set

In [51]:
cnn.fit(x=training_set, validation_data=test_set, epochs = 25) # try different epochs


# Part 4 - Making a single prediction

In [52]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img(path="../input/single-prediction/cat_or_dog_1.jpg",
                            target_size=(64,64))
# ! Important! - image have to have the same size as used during the training

#We need to convert the image into the array (because the prediction required the 2D array)
test_image=image.img_to_array(test_image)

# add an extra "fake" dimension: batch_size
test_image=np.expand_dims(test_image, axis=0) # having batch as the first dimension

result = cnn.predict(test_image)

# how to differentiate what is a dog(1) or a cat(0)

training_set.class_indices

if result[0][0]==1:
    prediction = "dog"
else:
    prediction = "cat"
#first accessing the batch and then accessing the single element of the batch

In [53]:
print(prediction)